#Base de datos limpia después de EDA


In [5]:
import pandas as pd
df = pd.read_csv('data_limpia.csv')
df.head()


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


#Extracción de Características

In [13]:

# Ejemplo de ingeniería de variables:
# Promedio de notas de 1er y 2do semestre
df['mean_grade'] = df[['Curricular units 1st sem (grade)', 'Curricular units 2nd sem (grade)']].mean(axis=1)

# Total de unidades curriculares aprobadas
df['total_approved'] = df['Curricular units 1st sem (approved)'] + df['Curricular units 2nd sem (approved)']

# Codificación de variables categóricas relevantes
cat_vars = ['Gender', 'Scholarship holder', 'International', 'Debtor', 'Displaced', 'Marital status']
df = pd.get_dummies(df, columns=cat_vars, drop_first=True)

# Codificación de la variable objetivo (Dropout=1, otros=0)
df['Target_bin'] = (df['Target'].str.lower().str.contains('drop')).astype(int)

df_limpio = df.copy()  # O el DataFrame que quieras guardar
df_limpio.to_csv('/content/data_features.csv', index=False)

#Modelamiento


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Cargar datos con features
df = pd.read_csv('data/processed/data_features.csv')

# Variables predictoras y objetivo
X = df.drop(['Target', 'Target_bin'], axis=1)
y = df['Target_bin']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)

# Línea base: Regresión Logística
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

# Modelo avanzado: Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Importancia de variables
importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("Top 10 features:\n", importances.head(10))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.8632768361581921
              precision    recall  f1-score   support

           0       0.87      0.94      0.90       601
           1       0.85      0.70      0.77       284

    accuracy                           0.86       885
   macro avg       0.86      0.82      0.84       885
weighted avg       0.86      0.86      0.86       885

Random Forest Accuracy: 0.8790960451977401
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       601
           1       0.86      0.74      0.80       284

    accuracy                           0.88       885
   macro avg       0.87      0.84      0.86       885
weighted avg       0.88      0.88      0.88       885

Top 10 features:
 total_approved                         0.119161
Curricular units 2nd sem (approved)    0.100722
mean_grade                             0.082145
Curricular units 2nd sem (grade)       0.074115
Curricular units 1st sem (approved)    0.065